# Implement KNN from scratch

The test problem we will be using in this tutorial is the iris classification.

The problem consists of 150 observations of iris flowers from three different species. There are 4 measurements of given flowers: sepal length, sepal width, petal length, and petal width (all in the same unit of centimeters). The predicted attribute is the species, which is either Setosa, Versicolor, or Virginica.

It is a standard dataset where the species is known for all instances. As such we can split the data into training and test datasets and use the results to evaluate our algorithm implementation. Good classification accuracy on this problem is above 90% correct (typically 96% or better).

Save the file in your current working directory with the file name “iris.data“.

## 1. Handle Data

In [7]:
#The first thing we need to do is load our data file

import csv
with open('data/iris.data', 'r') as csvfile:
    lines = csv.reader(csvfile)
    for row in lines :
        print (', '.join(row))
    

5.1, 3.5, 1.4, 0.2, Iris-setosa
4.9, 3.0, 1.4, 0.2, Iris-setosa
4.7, 3.2, 1.3, 0.2, Iris-setosa
4.6, 3.1, 1.5, 0.2, Iris-setosa
5.0, 3.6, 1.4, 0.2, Iris-setosa
5.4, 3.9, 1.7, 0.4, Iris-setosa
4.6, 3.4, 1.4, 0.3, Iris-setosa
5.0, 3.4, 1.5, 0.2, Iris-setosa
4.4, 2.9, 1.4, 0.2, Iris-setosa
4.9, 3.1, 1.5, 0.1, Iris-setosa
5.4, 3.7, 1.5, 0.2, Iris-setosa
4.8, 3.4, 1.6, 0.2, Iris-setosa
4.8, 3.0, 1.4, 0.1, Iris-setosa
4.3, 3.0, 1.1, 0.1, Iris-setosa
5.8, 4.0, 1.2, 0.2, Iris-setosa
5.7, 4.4, 1.5, 0.4, Iris-setosa
5.4, 3.9, 1.3, 0.4, Iris-setosa
5.1, 3.5, 1.4, 0.3, Iris-setosa
5.7, 3.8, 1.7, 0.3, Iris-setosa
5.1, 3.8, 1.5, 0.3, Iris-setosa
5.4, 3.4, 1.7, 0.2, Iris-setosa
5.1, 3.7, 1.5, 0.4, Iris-setosa
4.6, 3.6, 1.0, 0.2, Iris-setosa
5.1, 3.3, 1.7, 0.5, Iris-setosa
4.8, 3.4, 1.9, 0.2, Iris-setosa
5.0, 3.0, 1.6, 0.2, Iris-setosa
5.0, 3.4, 1.6, 0.4, Iris-setosa
5.2, 3.5, 1.5, 0.2, Iris-setosa
5.2, 3.4, 1.4, 0.2, Iris-setosa
4.7, 3.2, 1.6, 0.2, Iris-setosa
4.8, 3.1, 1.6, 0.2, Iris-setosa
5.4, 3.4

In [3]:
#Next we need to split the data into a training dataset

import random

def loadDataset(filename, split, trainingSet=[] , testSet=[]):
    with open(filename, 'r') as csvfile:
        lines = csv.reader(csvfile)
        dataset = list(lines)
        #parcourir les lignes
        for x in range(len(dataset)-1):
            #parcourir les colones
            for y in range(4):
                #convertir les string en float
                dataset[x][y] = float(dataset[x][y])
            if random.random() < split:
                #complete code
                trainingSet.append(dataset[x])
            else:
                #complete code
                testSet.append(dataset[x])

In [9]:
# test this function out with our iris dataset

trainingSet=[]
testSet=[]

loadDataset('data/iris.data', 0.66, trainingSet, testSet)
print ('Train: ' + repr(len(trainingSet)))
print ('Test: ' + repr(len(testSet)) )

Train: 93
Test: 56


## 2. Similarity

To make predictions we need to calculate the similarity between any two given data instances. This is needed so that we can locate the k most similar data instances in the training dataset for a given member of the test dataset and in turn, make a prediction.

Given that all four flower measurements are numeric and have the same units, we can directly use the Euclidean distance measure. 

Additionally, we want to control which fields to include in the distance calculation. Specifically, we only want to include the first 4 attributes. One approach is to limit the Euclidean distance to a fixed length, ignoring the final dimension.

In [74]:
# define the Euclidean distance
import math

def euclideanDistance(instance1, instance2, length):
    #Complete the function
    #The number of elements in the instance1 equals the number of elements in the instance2 
    #The length refers to the number of elements in the instance1 
    distance = 0
    for i in range(length):
       distance += math.pow(instance1[i]-instance2[i],2) 
    distance =  math.sqrt(distance)
    return distance

In [75]:
#test this function with some sample data
data1 = [2, 2, 2, 'a']
data2 = [4, 4, 4, 'b']
distance = euclideanDistance(data1, data2, 3)

print ('Distance: '+ repr(distance)) 

Distance: 3.4641016151377544


## 3. Neighbors

Now that we have a similarity measure, we can use it to collect the k most similar instances for a given unseen instance.

This is a straightforward process of calculating the distance for all instances and selecting a subset with the smallest distance values.

Below is the getNeighbors function that returns k most similar neighbors from the training set for a given test instance (using the already defined euclideanDistance function)

In [76]:
import operator

def getNeighbors(trainingSet, testInstance, k):
    distances = []
    length = len(testInstance)-1
    for x in range(len(trainingSet)):
        dist = euclideanDistance(testInstance, trainingSet[x], length)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

In [77]:
# test out this function :

trainSet = [[2, 2, 2, 'a'], [4, 4, 4, 'b']]
testInstance = [5, 5, 5]

k = 1

neighbors = getNeighbors(trainSet, testInstance, k)
print(neighbors)

[[4, 4, 4, 'b']]


## 4. Response

Once we have located the most similar neighbors for a test instance, the next task is to devise a predicted response based on those neighbors.

We can do this by allowing each neighbor to vote for their class attribute, and taking the majority vote as the prediction.

Below is a function for getting the majority voted response from a number of neighbors. It assumes the class is the last attribute for each neighbor.

In [78]:
import operator

def getResponse(neighbors):
    y = len(neighbors[0]) - 1 #added
    classVotes = {}
    for x in range(len(neighbors)):
        response = neighbors[x][y] #complete with appropriate number
        if response in classVotes:
            #Complete the if clause
            classVotes[response] += 1
        else:
            classVotes[response] = 1

        sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
    return sortedVotes[0][0]

In [79]:
#test out this function with some test neighbors:

neighbors = [[1,1,1,'a'], [2,2,2,'a'], [3,3,3,'b']]
response = getResponse(neighbors)
print(response)

a


This approach returns one response in the case of a draw, but you could handle such cases in a specific way, such as returning no response or selecting an unbiased random response.

## 5. Accuracy

We have all of the pieces of the KNN algorithm in place. An important remaining concern is how to evaluate the accuracy of predictions.

An easy way to evaluate the accuracy of the model is to calculate a ratio of the total correct predictions out of all predictions made, called classification accuracy.

Below is the getAccuracy function that sums the total correct predictions and returns the accuracy as a percentage of correct classifications.

In [80]:
def getAccuracy(testSet, predictions):
    #Complete the function
    #Takes in a test set and a predictions set
    #Returns the accuracy of the predictions
    #Assumes that the test set and predictions are both lists
    #Assumes that the lengths of the two lists are equal
    correct = 0.0
    y = len(testSet[0])-1
    #parcourir les lignes
    for x in range(len(testSet)-1):
        if (testSet[x][y] == predictions[x]):
            correct += 1.0
    
    return (correct/float(len(testSet))) * 100.0

In [81]:
#test this function with a test dataset and predictions:

testSet = [[1,1,1,'a'], [2,2,2,'a'], [3,3,3,'b']]
predictions = ['a', 'a', 'a']

accuracy = getAccuracy(testSet, predictions)
print(accuracy)

66.66666666666666


## 6. Main

We now have all the elements of the algorithm and  you can put them all in one main function

In [86]:
# load the iris dataset and split the data into a training dataset and a test dataset
trainingSet=[]
testSet=[]
loadDataset('iris.data', 0.75, trainingSet, testSet)

In [87]:
def KNearestNeighbors(trainingSet, k):
    #get list of predictions for each test instance
    predictions = []
    for testInstance in testSet:
        #get list of neighbors for each test instance
        neighbors = getNeighbors(trainingSet, testInstance, k)
        predictions.append(getResponse(neighbors))

    return predictions

In [88]:
# choose number of neigbors (k)
k = 3
predictions = KNearestNeighbors(trainingSet, k)

In [89]:
predictions

['Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-virginica',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-virginica',
 'Iris-virginica',
 'Iris-virginica',
 'Iris-virginica',
 'Iris-versicolor',
 'Iris-virginica',
 'Iris-virginica',
 'Iris-virginica',
 'Iris-virginica']

In [90]:
#get accuracy
accuracy = getAccuracy(testSet, predictions)
print(accuracy)

90.9090909090909


## 7. Another distance metric

In this part, you are asked to define another distance metric in addition to the Euclidean distance.

In [91]:
# define the Manhattan distance function
def manhattanDistance(instance1, instance2, length): 
    distance = 0
    for i in range(length):
        distance += abs(instance1[i]-instance2[i])
    
    return distance


In [92]:
# redifine getNeighbors so that it includes the distance function
def getNeighbors(trainingSet, testInstance, k, dist_func):
    distances = []
    length = len(testInstance)-1
    for x in range(len(trainingSet)):
        dist = dist_func(testInstance, trainingSet[x], length)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

In [93]:
# test the passing functions
trainSet = [[2, 2, 2, 'a'], [4, 4, 4, 'b']]
testInstance = [5, 5, 5]

neighbors = getNeighbors(trainSet, testInstance, 1, euclideanDistance)
print(neighbors)

[[4, 4, 4, 'b']]


In [94]:
neighbors = getNeighbors(trainSet, testInstance, 1, manhattanDistance)
print(neighbors)

[[4, 4, 4, 'b']]


In [95]:
# the new k nearest neighbor function
def KNearestNeighbors(trainingSet, k, dist_func):
    #get list of predictions for each test instance
    predictions = []
    for testInstance in testSet:
        #get list of neighbors for each test instance
        neighbors = getNeighbors(trainingSet, testInstance, k, dist_func)
        predictions.append(getResponse(neighbors))

    return predictions

In [96]:
# test knn with manhattan distance

# load the iris dataset and split the data into a training dataset and a test dataset
trainingSet=[]
testSet=[]
loadDataset('iris.data', 0.75, trainingSet, testSet)

# choose number of neigbors (k)
k = 3
predictions = KNearestNeighbors(trainingSet, k, manhattanDistance)

#get accuracy
accuracy = getAccuracy(testSet, predictions)
print(accuracy)

95.23809523809523
